In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用FastAPI和Vertex AI自定义容器服务部署虹膜检测模型

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/SDK_Custom_Container_Prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2FSDK_Custom_Container_Prediction.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab企业版中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/SDK_Custom_Container_Prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/SDK_Custom_Container_Prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

在本教程中，您将使用自定义容器方法在 Vertex AI 上构建一个 scikit-learn 模型并部署它。您将使用 FastAPI Python web 服务器框架创建一个预测端点。您还将把训练管道中的预处理器整合到您的在线服务应用程序中。

了解更多关于[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)和[Vertex AI 预测](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)。

### 目标

在这个笔记本中，您将学习如何在 Vertex AI 上创建、部署和提供一个自定义分类模型。这个笔记本更侧重于部署模型而不是模型本身的设计。


本教程使用以下 Vertex AI 服务和资源：

- Vertex AI 模型
- Vertex AI 终端

执行的步骤包括：

- 训练一个使用花朵测量值作为输入来预测鸢尾花类别的模型。
- 保存模型及其序列化的预处理器。
- 构建一个 FastAPI 服务器来处理预测和健康检查。
- 构建一个包含模型工件的自定义容器。
- 上传并部署自定义容器到 Vertex AI 终端。

### 数据集

本教程使用了R.A. Fisher的鸢尾花数据集，这是一个用于机器学习实验的小而受欢迎的数据集。每个实例具有四个数值特征，这些特征是花的不同测量值，还有一个目标标签，将花分类为：**鸢尾谷物**、**鸢尾变色**和**鸢尾维吉尼亚**。

本教程使用了[scikit-learn库中提供的一个版本的鸢尾花数据集](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris)。

成本

这个教程使用了 Google Cloud 的可计费组件：

* Vertex AI
* Cloud Storage
* Artifact Registry
* Cloud Build

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage 价格](https://cloud.google.com/storage/pricing)、[Artifact Registry 价格](https://cloud.google.com/artifact-registry/pricing) 和 [Cloud Build 价格](https://cloud.google.com/build/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

开始吧

### 安装 Python 的 Vertex AI SDK 和其他所需的软件包

将构建容器所需的要求写入文件中。

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev

安装依赖项。

In [ ]:
# Required in Docker serving container
! pip3 install -U  -r requirements.txt -q

# For local FastAPI development and running
! pip3 install -U  "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63 -q

# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform

### 重新启动运行时（仅限Colab）

要使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。请等待直到完成再继续下一步⚠️</b>
</div>

### 验证您的笔记本环境（仅适用于Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息
了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚未存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 初始化 Vertex AI SDK for Python

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目，并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

导入所需的库

In [ ]:
import os
import sys

### 配置资源名称

为以下参数设置名称：

`MODEL_ARTIFACT_DIR` - 在云存储桶内指向模型工件的文件夹路径，例如: "my-models/fraud-detection/trial-4"

`REPOSITORY` - 要创建或使用的工件库的名称。

`IMAGE` - 要推送到仓库的容器映像的名称。

`MODEL_DISPLAY_NAME` - Vertex AI 模型资源的显示名称。

In [ ]:
MODEL_ARTIFACT_DIR = "[your-artifact-directory]"  # @param {type:"string"}
REPOSITORY = "[your-repository-name]"  # @param {type:"string"}
IMAGE = "[your-image-name]"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "[your-model-display-name]"  # @param {type:"string"}

# Set the defaults if no names were specified
if MODEL_ARTIFACT_DIR == "[your-artifact-directory]":
    MODEL_ARTIFACT_DIR = "custom-container-prediction-model"

if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-container-prediction"

if IMAGE == "[your-image-name]":
    IMAGE = "sklearn-fastapi-server"

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "sklearn-custom-container"

编写您的预处理程序
标准化训练数据，使得每个数值特征列的平均值为0，标准差为1[可以改善您的模型](https://developers.google.com/machine-learning/crash-course/representation/cleaning-data)。

定义一个 `app` 文件夹并创建 `preprocess.py`，其中包含一个执行标准化操作的类。

In [ ]:
%mkdir app

In [ ]:
%%writefile app/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds


## 使用预处理器训练和存储模型
使用`preprocess.MySimpleScaler`对鸢尾花数据进行预处理，然后使用scikit-learn训练模型。

训练完成后，将训练好的模型导出为一个joblib（.joblib）文件，并将您的`MySimpleScaler`实例导出为一个pickle（.pkl）文件。

In [ ]:
%cd app/

import pickle

import joblib
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(model, "model.joblib")
with open("preprocessor.pkl", "wb") as f:
    pickle.dump(scaler, f)

### 将模型工件和自定义代码上传至云存储

在部署模型进行服务之前，Vertex AI 需要访问以下文件在云存储中：

* `model.joblib`（模型工件）
* `preprocessor.pkl`（模型工件）

运行以下命令来上传您的文件：

In [ ]:
!gsutil cp model.joblib preprocessor.pkl {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
%cd ..

构建一个FastAPI服务器

为了提供分类模型的预测结果, 构建一个FastAPI服务器应用程序。

In [ ]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import joblib
import json
import numpy as np
import pickle
import os

from google.cloud import storage
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris


app = FastAPI()
gcs_client = storage.Client()

with open("preprocessor.pkl", 'wb') as preprocessor_f, open("model.joblib", 'wb') as model_f:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/preprocessor.pkl", preprocessor_f
    )
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/model.joblib", model_f
    )

with open("preprocessor.pkl", "rb") as f:
    preprocessor = pickle.load(f)

_class_names = load_iris().target_names
_model = joblib.load("model.joblib")
_preprocessor = preprocessor


@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    inputs = np.asarray(instances)
    preprocessed_inputs = _preprocessor.preprocess(inputs)
    outputs = _model.predict(preprocessed_inputs)

    return {"predictions": [_class_names[class_num] for class_num in outputs]}


### 添加预启动脚本
FastAPI 在启动服务器之前执行以下脚本。将环境变量`PORT`设置为`AIP_HTTP_PORT`以运行FastAPI服务器。

In [ ]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

### 创建测试实例
要了解如何以JSON格式编写输入实例，请[阅读文档。](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
%%writefile instances.json
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

将容器镜像推送到 Artifact Registry。

使用`tiangolo/uvicorn-gunicorn-fastapi`作为基础镜像，编写`Dockerfile`。这将自动使用Gunicorn和Uvicorn为您运行FastAPI。访问[使用Docker部署FastAPI的文档](https://fastapi.tiangolo.com/deployment/docker/)获取更多信息。

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

### 本地测试图像（可选）

#### 在本地构建镜像（可选）

使用docker构建镜像以在本地测试。

**注意：** 在本教程中，docker仅用于本地测试容器。要部署到Artifact Registry，则使用Cloud-Build。

In [ ]:
IS_COLAB = "google.colab" in sys.modules

if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! sudo docker build \
            --tag="{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}" \
            .

### 在本地运行和测试容器（可选）

尝试以分离模式在本地运行容器，并提供容器所需的环境变量。这些变量将在 Vertex AI 部署后由 Vertex AI 提供给容器。测试 `/health` 和 `/predict` 路由，然后停止运行的镜像。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! sudo docker stop local-iris
    ! sudo docker rm local-iris
    ! docker run -d -p 80:8080 \
            --name=local-iris \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI={BUCKET_URI}/{MODEL_ARTIFACT_DIR} \
            "{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"

评估健康路径。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! curl localhost/health

传递`instances.json`并测试预测路径。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! curl -X POST \
      -d @instances.json \
      -H "Content-Type: application/json; charset=utf-8" \
      localhost/predict

停止并在本地删除容器。

In [ ]:
if not IS_COLAB and not os.getenv("IS_TESTING"):
    ! sudo docker stop local-iris
    ! sudo docker rm local-iris

在Artifact Registry中创建一个存储库

####启用Artifact Registry API
您必须为您的项目启用Artifact Registry API服务。

<a href="https://cloud.google.com/artifact-registry/docs/enable-service">了解更多关于启用服务的信息</a>。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

在Artifact Registry中创建一个Docker仓库
您的第一步是在Google Artifact Registry中创建自己的Docker仓库。

1 - 运行`gcloud artifacts repositories create`命令，使用您的区域和描述"docker仓库"来创建一个新的Docker仓库。

2 - 运行`gcloud artifacts repositories list`命令来验证您的仓库是否已创建。

In [ ]:
REPOSITORY = "my-docker-repo-unique"

! gcloud artifacts repositories create {REPOSITORY} --repository-format=docker --location={LOCATION} --description="Docker repository"

! gcloud artifacts repositories list

### 提交图像
使用 Cloud build 将图像推送到创建的构件存储库。

**注意:** 以下命令会自动考虑来自正在运行的目录的 Dockerfile。

In [ ]:
!gcloud builds submit --region={LOCATION} --tag={LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

部署到Vertex AI

使用物件 URI 创建 Vertex AI 模型
使用 Python SDK 在 Vertex AI 中上传您的模型物件。

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
)

### 部署模型到 Vertex AI Endpoint

一旦部署过程完成，模型将被部署并准备好在线提供预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

## 请求预测

向终端发送在线请求并获取预测结果。

### 使用Python SDK

使用Python SDK 从终端获取一个示例输入的预测结果。

In [ ]:
# Send some sample data to the endpoint
endpoint.predict(instances=[[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]])

使用REST

使用curl请求从端点获取预测。

In [ ]:
# Fetch the endpoint name
ENDPOINT_ID = endpoint.name

In [ ]:
# Send a prediction request using sample data 
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{LOCATION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/{ENDPOINT_ID}:predict

使用gcloud CLI从端点获取预测。

In [ ]:
!gcloud ai endpoints predict $ENDPOINT_ID \
  --region=$LOCATION \
  --json-request=instances.json

清理

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 模型
- 端点
- Artifact Registry镜像
- Artifact存储库：将`delete_art_repo`设置为**True**以删除本教程中创建的存储库。
- Cloud Storage存储桶：将`delete_bucket`设置为**True**以删除本教程中使用的Cloud Storage存储桶。

In [ ]:
delete_bucket = False
delete_art_repo = False
    
# Undeploy model and delete endpoint
endpoint.undeploy_all()
endpoint.delete()

#Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {LOCATION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

# Delete the Artifact Repository
if delete_art_repo:
    ! gcloud artifacts repositories delete {REPOSITORY} --location=$LOCATION -q
    
# Delete the Cloud Storage bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI

清理本地创建的文件。

In [ ]:
! rm -rf app/
! rm requirements.txt
! rm instances.json
! rm Dockerfile